In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy
import skimage
import ImageD11
import ImageD11.grain
import ImageD11.columnfile
import ImageD11.unitcell
import xfab
import xfab.symmetry
import xfab.tools
from scipy.ndimage import median_filter, binary_fill_holes, label
xfab.CHECKS.activated = False

from mpl_toolkits.mplot3d import Axes3D
from itertools import product, combinations

In [ ]:
parfile = '/home/ax8427he/workspace/AL_DTU/data/experiment/sam3/layer_bottom/aluS3.par'
params = ImageD11.parameters.read_par_file(parfile)
dzero_cell = [params.get('cell_'+key) for key in ('_a','_b','_c','alpha','beta','gamma')]
B0 = xfab.tools.form_b_mat(dzero_cell) / (np.pi*2)

In [ ]:


# https://next-gen.materialsproject.org/materials/mp-134
D_Al = np.array( [
        [  104,   73, 	  73,    0,    0, 	 0   ],
        [  73, 	 104, 	  73, 	 0,    0, 	 0   ],
        [  73, 	  73, 	 104, 	 0,    0, 	 0   ],
        [  0, 	  0, 	  0, 	32,    0,    0   ],
        [  0, 	  0, 	  0, 	 0,    32, 	 0   ],
        [  0, 	  0, 	  0, 	 0,    0,    32  ]] ) # units of GPa     

C_c = np.linalg.inv(B0).T
E  = np.column_stack((C_c[:, 0], np.cross(C_c[:, 2], C_c[:, 0]), C_c[:, 2]))
E /= np.linalg.norm(E, axis=0)

def to_tensor( s ):
    e11, e22, e33, e12, e13, e23 = s
    return np.array([[e11,e12,e13],[e12,e22,e23],[e13,e23,e33]])

def to_list( s ):
    return np.array( [s[0,0], s[1,1], s[2,2], s[0,1], s[0,2], s[1,2]] )

def get_stress_Fe_elastic( strain_sample_tensor, U ):
    strain_crystal_tensor =  E.T @( U.T @ strain_sample_tensor @ U ) @ E    
    e11, e22, e33, e12, e13, e23 = to_list( strain_crystal_tensor )
    strain_crystal_voigt = np.array( [e11, e22, e33, 2*e23, 2*e13, 2*e12] )
    stess_crystal_voigt = D_Al @ strain_crystal_voigt
    s11, s22, s33, s23, s13, s12 = stess_crystal_voigt
    stess_crystal = np.array( [s11, s22, s33, s12, s13, s23] )
    stess_crystal_tensor = to_tensor( stess_crystal )
    stess_sample_tensor = U @ ( E @ stess_crystal_tensor @ E.T ) @ U.T
    return stess_sample_tensor # note that the stress is now in GPa!


In [ ]:
X     = np.load('X_coord_gmap.npy')
Y     = np.load('Y_coord_gmap.npy')
sample = np.fliplr(  np.load('sample_mask_layer_bottom.npy')[2:-2, 2:-2] )
ubi_map = np.load('ubi_median_filt_s6.npy', allow_pickle=True ) 
ipf = np.load('ipf_median_filt_s6.npy', allow_pickle=True ) 

In [ ]:
refined_ubi_map_test_strain_0 = np.load('refined_ubi_and_strain_map_com_corrected_test.npy', allow_pickle=True)

In [ ]:
strain_refined = np.zeros_like(ubi_map)
for i in range(refined_ubi_map_test_strain_0.shape[0]):
    for j in range(refined_ubi_map_test_strain_0.shape[1]):
        ubis, strain, npks = refined_ubi_map_test_strain_0[i,j]
        refubi = ubi_map[i,j]
        if len(npks)>0:
            scores = [np.abs(refubi-ubi).sum() for ubi in ubis]
            strain_refined[i,j] = strain[np.argmin(scores)]


In [ ]:
stress = np.zeros_like(strain_refined)
for i in range(stress.shape[0]):
    for j in range(stress.shape[1]):
        if sample[i,j]:
            U = ImageD11.grain.grain(ubi_map[i,j]).u
            strain_sample_tensor = strain_refined[i,j]
            stress[i,j] = get_stress_Fe_elastic( strain_sample_tensor, U = U )*1e9

In [ ]:
np.save('stress_map_bottom_layer_refined_com_corrected.npy', stress)
np.save('strain_map_bottom_layer_refined_com_corrected.npy', strain_refined)